In [4]:
%pip install -qU langchain_community yfinance

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from langchain_community.llms import Ollama
import pprint
import yfinance

In [2]:
AGENT_MODEL_NAME = "arcee-ai/arcee-agent"

In [3]:
agent = Ollama(
    model = AGENT_MODEL_NAME,
    system = """You are a helpful chat assistant""",
    top_p = 0.7,
)

In [5]:
func_prompt = """You have four primary functions: checking the last price of a specified stock, finding the name of a company CEO, finding what a company does, and answering specific questions about a company. Use the appropriate function based on the user's query.

### Functions:

1. **get_stock_price(company_name: str, stock_symbol: str) -> str**
    - This function returns the last close price of a specified stock.
    - Input: A company name (e.g., Mc Donalds), which you must convert to a stock symbol (e.g., MCD).
    - Output: A string containing the last close price of the specified stock (e.g., "The last closing price of Mc Donalds (MCD) is $250.00").

2. **get_ceo_name(company_name: str, stock_symbol: str) -> str**
- This function returns the name of the CEO of a specified company.
- Input: A company name (e.g., Mc Donalds), which you must convert to a stock symbol (e.g., MCD).
- Output: A string containing the name of the CEO of the specified company (e.g., "The CEO of Mc Donalds is John Doe").

3. **get_company_summary(company_name: str, stock_symbol: str) -> str**
- This function returns a summary describing the business activities of a specified company.
- Input: A company name (e.g., Mc Donalds), which you must convert to a stock symbol (e.g., MCD).
- Output: A string containing a detailed summary of the specified company's business activities.

4. **answer_general_question(question: str) -> str**
    - This function answers questions in general.
    - Input: a user question.
    - Output: your best answer to the user question.

### Instructions:

- If the user asks a question related to the price of a stock, use the `get_stock_price` function.
- If the user asks a question related to the CEO of a company, use the `get_ceo_name` function.
- If the user asks a general question about a company's activities, use the `get_company_summary` function.
- If the user asks any other question, use the `answer_general_question` function. Only return the result of the function call, not your internal reasoning.
- Only return the result of the function call.

### User Query:\n"""

In [6]:
def get_stock_price(company_name, stock_symbol):
    stock = yfinance.Ticker(stock_symbol)
    price = stock.history(period="1d")["Close"].values[0]
    return f"The last closing price of {company_name} ({stock_symbol}) was ${price:.2f}."


def get_ceo_name(company_name, stock_symbol):
    stock = yfinance.Ticker(stock_symbol)
    info = stock.info
    ceo = info['companyOfficers'][0]['name']
    return f"The CEO of {company_name} is {ceo}. The full job title is {info['companyOfficers'][0]['title']}."


def get_company_summary(company_name, stock_symbol):
    stock = yfinance.Ticker(stock_symbol)
    summary = stock.info['longBusinessSummary']
    return f"{company_name} ({stock_symbol}) is a company that is involved in {summary}."


def answer_general_question(question):
    ans = agent.invoke(f"""{question}\n""")
    return ans

In [7]:
def function_call(input, model=agent, func_prompt=func_prompt):
    """
    Interacts with an AI model by combining user input with a custom prompt.

    Args:
        input (str): The user's input string.
        model (object, optional): The AI model to invoke. Defaults to agent.
        func_prompt (str, optional): The custom prompt to combine with the user's input. Defaults to func_prompt.

    Returns:
        str: The response generated by the AI model in response to the combined prompt and input.
    """
    payload = f"""{func_prompt}{input}\n"""
    ans = model.invoke(payload)
    return ans

In [8]:
def llm_pack(input):
    """
    Evaluates a string generated by an AI model and returns the result.

    This function wraps around 'function_call' to convert the AI's response into a executable code snippet,
    executes it, and then returns the result. If any exception occurs during execution, it returns None.

    Args:
        input (str): The input to pass to 'function_call'.

    Returns:
        str: The result of the function call.
    """
    try:
        func = function_call(input).strip()
        print(func)
        code = f"result = {func}"
        local_vars = {}
        exec(code, globals(), local_vars)
        ans = local_vars.get('result').strip()
        return ans
    except Exception as e:
        return

In [9]:
response = llm_pack("What the stock price for Tesla?")
print(response)

get_stock_price("Tesla", "TSLA")
The last closing price of Tesla (TSLA) was $191.76.


In [10]:
response = llm_pack("Who runs Tesla?")
print(response)

get_ceo_name("Tesla", "TSLA")
The CEO of Tesla is Mr. Elon R. Musk. The full job title is Co-Founder, Technoking of Tesla, CEO & Director.


In [11]:
response = llm_pack("What does Tesla build?")
pprint.pprint(response)

get_company_summary("Tesla", "TSLA")
('Tesla (TSLA) is a company that is involved in Tesla, Inc. designs, develops, '
 'manufactures, leases, and sells electric vehicles, and energy generation and '
 'storage systems in the United States, China, and internationally. The '
 'company operates in two segments, Automotive, and Energy Generation and '
 'Storage. The Automotive segment offers electric vehicles, as well as sells '
 'automotive regulatory credits; and non-warranty after-sales vehicle, used '
 'vehicles, body shop and parts, supercharging, retail merchandise, and '
 'vehicle insurance services. This segment also provides sedans and sport '
 'utility vehicles through direct and used vehicle sales, a network of Tesla '
 'Superchargers, and in-app upgrades; purchase financing and leasing services; '
 'services for electric vehicles through its company-owned service locations '
 'and Tesla mobile service technicians; and vehicle limited warranties and '
 'extended service plans. Th

In [12]:
response = llm_pack("Who are the main competitors of Tesla?")
print(response)

answer_general_question("Who are the main competitors of Tesla?")
Tesla's primary competitors in the electric vehicle (EV) market include:

1. **Nissan Leaf**: Nissan's first mass-produced EV, offering an affordable and well-established option for consumers.
2. **Chevrolet Bolt**: A longer-range EV from General Motors, targeting the same market segment as Tesla's Model 3.
3. **Kia Niro EV**: A compact crossover SUV with competitive range and features at a lower price point than some of Tesla's models.
4. **Volkswagen ID.4**: Volkswagen's first fully electric vehicle, designed to challenge Tesla in the mid-size SUV market.
5. **BMW iX3**: BMW's entry into the luxury EV segment, offering performance and features similar to Tesla's vehicles.
6. **Mercedes-Benz EQC**: Mercedes' first all-electric SUV, targeting a premium customer base with advanced technology and design.
7. **Ford Mustang Mach-E**: A sporty crossover SUV that competes directly with Tesla's Model Y in terms of style and per